In [22]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
import sys
sys.path.insert(0, '../')
from livecell_tracker import core
from livecell_tracker.core import datasets
from livecell_tracker.core.datasets import LiveCellImageDataset, SingleImageDataset
from skimage import measure
from livecell_tracker.core import SingleCellTrajectory, SingleCellStatic
import numpy as np
import os, json, cv2, random
import cv2

## SingleCellStatic: saving & loading 

In [23]:
io_out_dir = Path("test_io_output")

In [24]:
dataset_dir_path = Path(
    "../datasets/test_data_STAV-A549/DIC_data"
)

mask_dataset_path = Path("../datasets/test_data_STAV-A549/mask_data")


In [25]:
mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="png")

time2url = sorted(glob.glob(str((Path(dataset_dir_path) / Path("*_DIC.tif")))))
time2url = {i: path for i, path in enumerate(time2url)}
dic_dataset = LiveCellImageDataset(time2url=time2url, ext="tif")

3 png img file paths loaded;


In [26]:
from skimage.measure import regionprops
from livecell_tracker.segment.utils import prep_scs_from_mask_dataset
single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)

100%|██████████| 3/3 [00:11<00:00,  3.76s/it]


In [27]:
single_cells[1].meta

{}

In [28]:
sc_json_list = SingleCellStatic.write_single_cells_json(single_cells, io_out_dir/"single_cells.json", dataset_dir=io_out_dir/"dataset", return_list=True)

In [29]:
SingleCellStatic.write_single_cells_json(single_cells, io_out_dir/"single_cells.json", dataset_dir=io_out_dir/"dataset", return_list=False)

In [30]:
img_dataset = single_cells[0].img_dataset

In [31]:
loaded_scs = SingleCellStatic.load_single_cells_json(io_out_dir/"single_cells.json")

In [32]:
loaded_scs[0]

SingleCellStatic(id=2dfb924f-96ac-4969-9e40-bdab2165c652, timeframe=0, bbox=[ 143.  978.  207. 1044.])

In [33]:
for sc in single_cells[:2]:
    print(sc.id)

2dfb924f-96ac-4969-9e40-bdab2165c652
cd12571f-3d62-4373-9aeb-3be7a1ed4ce1


In [34]:
for sc in loaded_scs[:2]:
    print(sc.id)

2dfb924f-96ac-4969-9e40-bdab2165c652
cd12571f-3d62-4373-9aeb-3be7a1ed4ce1


In [35]:
for sc in single_cells:
    for loaded_sc in loaded_scs:
        if sc.id == loaded_sc.id:
            # compare contour, contours are np.array
            assert np.allclose(sc.contour, loaded_sc.contour), f"the difference between sc.contour and loaded_sc.contour is {sc.contour - loaded_sc.contour}, ids are {sc.id} and {loaded_sc.id}"

## SingleCellTrajectory: saving & loading